In [69]:
import csv
import json
import numpy as np
import pandas as pd
import re

from collections import Counter
from io import StringIO

In [70]:
CENSUS_DATA_LOCATION = 'build/ACS_14_5YR_B01003_with_ann.csv'
ELECTION_DATA_LOCATION = 'python/data.csv'
OUTPUT_LOCATION = 'build/merged_data.csv'
US_JSON_LOCATION = 'us.json'
OUTPUT_US_JSON_LOCATION = 'public/data/us.json'

df = pd.read_csv(ELECTION_DATA_LOCATION)

lines = []

with open(CENSUS_DATA_LOCATION, encoding='latin1') as f:
    reader = csv.reader(f)
    next(reader)
    for line in reader:
        lines.append([line[0], line[1], line[2], int(line[3])])

pop_df = pd.DataFrame.from_records(lines, columns=['id', 'id2', 'name', 'total'])

In [71]:
df.head()

,state,county,reporting,dem,gop,lib,grn,una,oth
0,AL,Autauga,100%,5908,18110,538,105,0,0
1,AL,Baldwin,100%,18409,72780,2448,453,0,0
2,AL,Barbour,100%,4848,5431,93,18,0,0
3,AL,Bibb,100%,1874,6733,124,17,0,0
4,AL,Blount,100%,2150,22808,337,89,0,0


In [72]:
pop_df.head()

,id,id2,name,total
0,0500000US01001,01001,"Autauga County, Alabama",55136
1,0500000US01003,01003,"Baldwin County, Alabama",191205
2,0500000US01005,01005,"Barbour County, Alabama",27119
3,0500000US01007,01007,"Bibb County, Alabama",22653
4,0500000US01009,01009,"Blount County, Alabama",57645


In [73]:
pop_df['county'] = [x[0] for x in pop_df.name.str.split(',')]
pop_df['state'] = [x[1] for x in pop_df.name.str.split(',')]

In [74]:
STATES = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

STATE_ABBREVS = [
  'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL',
  'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME',
  'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH',
  'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI',
  'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',
  'WY']

STATE_TO_ABBREV = dict(zip(STATES, STATE_ABBREVS))

assert len(STATE_TO_ABBREV) == 51

In [75]:
pop_df['state'] = np.array(pop_df.name.apply(lambda x: x.split(',')[1]))
pop_df['state'] =pop_df['state'].str.strip()
pop_df['state_abbrev'] = pop_df['state'].apply(lambda x: STATE_TO_ABBREV[x])

In [76]:
# All found corrections
df.loc[df['county'] == 'Bedford Co.', 'county'] = 'Bedford'
pop_df.loc[pop_df['name'] == 'Doña Ana County, New Mexico', 'name'] = 'Dona Ana County, New Mexico'
df.loc[df['county'] == 'Sainte Genevieve', 'county'] = 'Ste. Genevieve'

In [77]:
# Election data in AK only at state level. Combine....
ak_population = pop_df[pop_df['state_abbrev'] == 'AK'].total.sum()
ak_df = pd.DataFrame.from_records([(-1, -1, 'Alaska, Alaska', ak_population, 'Alaska', 'Alaska', 'AK')],
                                  columns=('id', 'id2', 'name', 'total', 'county', 'state', 'state_abbrev'))
pop_df = pop_df.loc[~(pop_df['state_abbrev'] == 'AK')]
pop_df = pd.concat([pop_df, ak_df])

In [78]:
pop_df[pop_df['state_abbrev'] == 'AK']

,id,id2,name,total,county,state,state_abbrev
0,-1,-1,"Alaska, Alaska",728300,Alaska,Alaska,AK


In [79]:
# Then there are a bunch of special cases.....
records_to_join_on = []

def row_with_county(row, county):
    records_to_join_on.append((row.id, row.id2, row.name, row.total,
                               county, row.state, row.state_abbrev))

for i, row in pop_df.iterrows():
    name = row['name']
    state = row.state_abbrev
    if name.startswith('Baltimore') and state == 'MD':
        if 'ity' in name:
            row_with_county(row, 'Baltimore City')
        else:
            row_with_county(row, 'Baltimore County')
    elif name.startswith('Carson City') and state == 'NV':
        row_with_county(row, 'Carson City')
    elif state == 'VA' and any(name.startswith(x) for x in ('Fairfax', 'Franklin', 'Richmond', 'Roanoke')):
        if 'County' in name:
            row_with_county(row, name.split(' ')[0] + ' Co.')
        else:
            row_with_county(row, name.split(' ')[0])
    elif name.startswith('LaSalle'):
        row_with_county(row, 'La Salle')
    elif state == 'DC':
        row_with_county(row, 'Washington')
    elif state == 'MO' and name.startswith('St. Louis'):
        if 'County' in name:
            row_with_county(row, 'St. Louis Co.')
        else:
            row_with_county(row, 'St. Louis')
    elif state == 'HI' and 'Kalawao' in name:
        # This is some uninhabited place
        pass
    elif state == 'SD' and 'Shannon' in name:
        # Name change in 2015
        row_with_county(row, 'Oglala Lakota')
    elif state == 'AK':
        row_with_county(row, 'Alaska')
    else:
        match = re.match(r'^(.*) (?=County|Parish|city)', name)
        if not match:
            print("WTF", row)
        else:
            row_with_county(row, match.groups()[0])

In [80]:
pop_df_fixed = pd.DataFrame.from_records(
    records_to_join_on, columns=['id', 'id2', 'name', 'total', 
                                 'county', 'state', 'state_abbrev'])

In [81]:
merged_df = df.merge(pop_df_fixed, how='inner', 
         left_on=['county', 'state'],
         right_on=['county', 'state_abbrev'])

assert len(merged_df) == len(df)

In [82]:
merged_df.rename(columns={'state_y': 'full_state', 'state_x': 'state'}, inplace=True)
del merged_df['state_abbrev']

In [83]:
merged_df['id2'] = np.array(['{:05d}'.format(x) for x in merged_df.id2])

ValueError: Unknown format code 'd' for object of type 'str'

In [84]:
merged_df.to_csv(OUTPUT_LOCATION, index=False)

In [85]:
color = {}
for state in 'AK, AL, AR, CT, DE, HI, IL, ME, MI, MN, MT, NE, NM, NV, SC, VA, WA'.split(', '):
    color[state] = 0
for state in 'AZ, DC, FL, KS, KY, MS, NC, ND, OR, PA, RI, TX, VT, WI, WY'.split(', '):
    color[state] = 1
for state in 'CA, CO, GA, ID, IN, LA, MA, MO, NJ, SD, WV'.split(', '):
    color[state] = 2
for state in 'IA, MD, NH, NY, OH, OK, TN, UT'.split(', '):
    color[state] = 3

id_to_properties = {int(row.id2): {'state': row.state,
                                   'color': color[row.state],
                                   'name': row.county, 
                                   'population': row.total,
                                   'dem': row.dem,
                                   'gop': row.gop, 
                                   'grn': row.grn,
                                   'lib': row.lib, 
                                   'oth': row.oth}
                   for _, row in merged_df.iterrows()}

In [86]:
with open(US_JSON_LOCATION, 'r') as f:
    data = json.load(f)

In [87]:
for val in data['objects']['counties']['geometries']:
    if val['id'] in id_to_properties:
        val['properties'] = id_to_properties[val['id']]

In [88]:
with open(OUTPUT_US_JSON_LOCATION, 'wt') as f:
    json.dump(data, f)

In [89]:
data['objects']['counties']['geometries'][0]

{'arcs': [[[0, 1, 2]]],
 'id': 53073,
 'properties': {'color': 0,
  'dem': 59402,
  'gop': 40247,
  'grn': 2954,
  'lib': 4745,
  'name': 'Whatcom',
  'oth': 875,
  'population': 204855,
  'state': 'WA'},
 'type': 'MultiPolygon'}